## Downloading and Extracting data sets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# get the crawled data from the smaller languages (yue)
!wget http://dumps.wikimedia.org/zh_yuewiki/20211020/zh_yuewiki-20211020-page.sql.gz
!wget http://dumps.wikimedia.org/zh_yuewiki/20211020/zh_yuewiki-20211020-langlinks.sql.gz

In [ ]:
# download the dataset
!wget https://dumps.wikimedia.org/zh_yuewiki/20211020/zh_yuewiki-20211020-pages-articles.xml.bz2

In [ ]:
!git clone https://github.com/clab/wikipedia-parallel-titles

In [ ]:
#extract parallel titles
!/content/wikipedia-parallel-titles/build-corpus.sh zh zh_yuewiki-20211020 > titles.txt

In [ ]:
#extracts and cleans text from a Wikipedia database backup dump
!pip install wikiextractor

In [ ]:
!pip install gensim

### YUE

In [ ]:
!python make_wiki_corpus.py zh_yuewiki-20211020-pages-articles.xml.bz2 wiki_yue.txt
# 15,410 articles

Processed 10000 articles
Processing complete!


In [ ]:
# see the word count of the wiki corpus of yue
!wc -c wiki_yue.txt | awk '{print $1}'

44069348


### ZH

In [ ]:
# get the crawled data from the Chinese language
!wget https://dumps.wikimedia.org/zhwiki/20211020/zhwiki-20211020-pages-articles.xml.bz2

In [ ]:
!python make_wiki_corpus.py zhwiki-20211020-pages-articles.xml.bz2 wiki_zh.txt

In [ ]:
!tar -czvf zhwiki_corpus.tar.gz wiki_zh.txt

wiki_zh.txt


In [ ]:
!wc -c wiki_zh.txt | awk '{print $1}'

95238399


## Data loading and prepossessing 

In [ ]:
# unzip the data file of ZH
!tar -xvf /content/drive/MyDrive/rd_data/zhwiki_corpus.tar.gz

In [1]:
def read_txt(in_file):
    with open(in_file, 'r', encoding="utf-8") as f:
       lines = [line.rstrip() for line in f]
    return lines

def article_to_list_of_words(in_list):
    # convert a string like variable to a list of tokens (words)
    return list(in_list.split(" "))

def batch_load_lists_of_words(input_data):
    # convert all articles to a nested list, each list containing a list of words in the file.
    one_articles = [article_to_list_of_words(each) for each in input_data]
    return one_articles

def fetch_titles(inlist):
    # get all the titles in the wikipedia crawl
    titles = [article_to_list_of_words(each)[0] for each in inlist]
    return titles

In [ ]:
!ls /content/drive/MyDrive/rd_data

wiki_yue.txt  zhwiki_corpus.tar.gz


In [ ]:
# read the data files
datadir = "/content/drive/MyDrive/rd_data/"
zh_path = datadir+"wiki_zh.txt"
yue_path = datadir+"wiki_yue.txt"

zhwiki = read_txt("wiki_zh.txt")
yuewiki = read_txt(yue_path)

# load the files from lists of strings to lists of lists of words
yue_list = batch_load_lists_of_words(yuewiki)
zh_list = batch_load_lists_of_words(zhwiki)
print(yue_list[0])   #spotcheck
print(zh_list[0])  #spotcheck

# Get a list of the titles only in both lanuuages
yue_titles = fetch_titles(yuewiki)
zh_titles = fetch_titles(zhwiki)
print(yue_titles[0])   #spotcheck
print(zh_titles[0])  #spotcheck

In [ ]:
print(f"There are {len(yuewiki)} articles in YUE wiki")
print(f"There are {len(zhwiki)} articles in ZH wiki")

There are 15410 articles in YUE wiki
There are 408936 articles in ZH wiki


In [ ]:
!pip install opencc

In [ ]:
yue_list[9222]

In [ ]:
# need to translate the titles in ZH from SC to TC
import opencc
converter = opencc.OpenCC('s2hk.json') # Simplified to Traditional Chinese (Hong Kong variant) 簡體到香港繁體
translated_ZHtitles = [converter.convert(title) for title in zh_titles]
print(translated_ZHtitles)

# find intersection between translated_ZHtitles and yue_titles
intersected_titles = set(translated_ZHtitles).intersection(set(yue_titles))
print(intersected_titles)
print(len(intersected_titles))

In [ ]:
print(intersected_titles)
print(len(intersected_titles))

{'語言學', '人心果', '屠呦呦', '廉政狙擊', '彩虹小馬', '地不佬', '歌聲妹影', '馬如風', '先天道', '洛溪站', '政府飛行服務隊', '彼得二世', '尖子攻略', 'se', '文沖站', '天市垣', '徐小鳳', '李司棋', '張發奎', '借宿一宵', 'soul', '黃耀明', '創世電視', 'do', '陳俞希', '葵涌循道中學', '死海', '曾俊華', '德川家康', '佛山祖廟', '金曲獎最佳客語專輯獎', '今生無悔', '澳門特別行政區', '香港理工大學', '廉政行動', '少年警訊', '廣州客車廠', '潘清簡', '糧船灣超級火山', '黃綺珊', '阿美利堅有外星人', '吳君如', '嶺南學派', '序言書室', '大叔的愛', '問誰未發聲', '妳的名字我的姓氏', '冥王聖幀', '江欣燕', 'pt', '何炅', '馬來西亞政府', '龔慈恩', '八點檔', '李克強', '假冒女團', '鹿兒島市', '香港海洋公園水上樂園', '海鳴威', '天主教基督勞工堂', '吳思佳', '邱士縉', '日伽', '香港紅十字會雅麗珊郡主學校', '藍港互動集團有限公司', '鍾晴', '何婉盈', '歡樂組', '荃灣碼頭', '伊朗', '亞羅士打', '議長', '長壽路', '娛樂', '楊愛瑾', '武則天', '深白色', '青玉案', '九龍站', '馮定國', '湯怡', '劉令飛', '名人廚房', '太古倉碼頭', '許鏡清', '慣性收視', '吳美雲', '康子內外篇', '馬來人至上', 'dog', '有人共鳴', '澄母', '薄扶林水塘', '在拓撲學及數學的其它相關領域', '李穎芝', 'ara', '李珣', '黃竹坑新圍舊民居', '恐鳥', '愛上一個不回家的人', '仙人掌', '天平架站', '漢堡王', 'intel', '遊戲', '澳洲華裔小姐競選', '防禦性編程', '男同性戀', '陳百祥', 'uwants討論區', '藍玉', '馬來西亞運動會', '陳鴻烈', '周禮茂', '山階宮晃親王', 'muse', '陳水扁', '任達華', '新中華飯店', '王嘉慧', '蔡展鵬', '香港中文大學工程學院', '畢

In [ ]:
def find_title_idx_in_texts(check, list_to_check):
  positions = ["{} {}".format(index1,index2) for index1,value1 in enumerate(list_to_check) for index2,value2 in enumerate(value1) if value2==check]
  nested_index_list = [list(pos.split(" ")) for pos in positions]
  title_index = [sublist for sublist in nested_index_list if '0' in sublist]
  if len(title_index) < 1:
      return None
  flattened_title_index =  [item for sublist in title_index for item in sublist]
  return int(flattened_title_index[0])

find_title_idx_in_texts(check="0", list_to_check=yue_list)

In [88]:
def create_dict(tgtstrings, tgtlist, tgtlang_code, srcstrings, srclist, srclang_code):
    lang_dict = {}
    # tgt lang
    lang_dict[tgtlang_code] = {}
    for idx, item in enumerate(tgtlist):
        wiki_pagename = item[0]
        if wiki_pagename in intersected_titles:
            lang_dict[tgtlang_code][wiki_pagename] = tgtstrings[idx]
    # src lang
    lang_dict[srclang_code] = {}
    for idx1, item1 in enumerate(srclist):
        wiki_pagename_src = item1[0]
        if wiki_pagename_src in intersected_titles:
            lang_dict[srclang_code][wiki_pagename_src] = srcstrings[idx1]
    return lang_dict
  
yuezh_dict = create_dict(yuewiki, yue_list, "yue", zhwiki, zh_list, "zh")

In [89]:
print(yuezh_dict.get("zh", {}).get('愛因斯坦'))
print(yuezh_dict.get("yue", {}).get('愛因斯坦'))

愛因斯坦 嘉當理論 此理論以物理學家阿爾伯特 愛因斯坦以及埃利 嘉當 為名 作為古典物理中的主要理論 廣義相對論卻有一個缺點 其無法描述 自旋軌道耦合 亦即內稟角動量 自旋 與軌道角動量 間的交換 存在有定量的理論證明 其顯示 當物體具有自旋性質時 廣義相對論必須要擴充成愛因斯坦 嘉當理論 實驗上的效應由於太小 目前尚無法觀測得到 歷史 爱因斯坦引力理论 以来 取得了巨大的成功 随着实验观察数据的积累 爱因斯坦引力理论遇到了许多困难 必须引入具有负压的暗能量 为了在爱因斯坦引力理论 牛顿引力理论 中说明此现象 必须引入占星系质量为 的暗物质 牛顿引力理论 牛顿引力理论 及其它物理效应说明的微小作用力 爱因斯坦引力理论具有缺陷 爱因斯坦引力理论可能存在缺陷 我们发现 因此我们有理由认为 爱因斯坦引力理论具有缺陷 嘉當引力 自旋场理论 因此可以认为愛因斯坦 嘉當引力 利用愛因斯坦 嘉當引力 也可以说明星系暗物质的分布情况 愛因斯坦 嘉當引力 自旋场理论预言 自旋作用力 動機 而廣義相對論是建構於其上 在黎曼幾何中 里奇曲率張量 ricci curvature tensor 必須是 對稱的 亦即 因此愛因斯坦曲率張量 einstein curvature tensor 定義為 也必須是對稱的 在廣義相對論中 且其 透過重力常數的聯繫 等同於應力 能量張量或能量 動量張量 此處我們將能量 動量張量表示為 動量張量的 在愛因斯坦 嘉當理論留給仿射扭率 affine torsion 然而 當自旋與軌道角動量進行交換時 根據角動量守恆的廣義式 則知動量張量為不對稱的 spin current 之散度 細節請參考 spin tensor 條目 埃利 affine torsion 其允許里奇張量可以是不對稱的 雖然自旋 愛因斯坦 嘉當理論則相當重要 因為 其顯示出仿射理論 而非度規理論 對於重力能提供更好的描述 其解釋仿射扭率的意義 在一些量子重力理論中自然出現 其將自旋詮釋為仿射扭率 在幾何意義上是時空介質 spacetime medium 之位錯場 field of dislocations 的一項連續近似 黎曼 嘉當幾何 riemann cartan geometry 幾何與表示式 affine differential geometry 其中我們賦予n維微分流形m 一微分流形

In [105]:
pdd1 = pd.DataFrame.from_dict(yuezh_dict)
pdd1.head(5)

,yue,zh
香港,香港 年度授勳同嘉獎名單 號喺憲報刊登 共有 人授勳同嘉獎 授勳典禮 號同 號喺禮賓府舉行 ...,香港 年度授勳及嘉獎名單 日於憲報刊登 授勳及嘉獎名單 大紫荊勳章 張舉能首席法官 鄭若驊司...
愛因斯坦,愛因斯坦 全名音譯 阿爾拔 愛因斯坦 係一位理論物理學家 佢最出名嘅係發表咗相對論 另外喺量...,愛因斯坦 嘉當理論 此理論以物理學家阿爾伯特 愛因斯坦以及埃利 嘉當 為名 作為古典物理中的...
北京,北京 普通話拼音 běijīng 音標 peɪ ɕiŋ 傳統英文名 peking 簡稱 係中...,北京 年冬残奥会代表团团长会召开 代表团团长会期间 北京冬奥组委体育 场馆与基础设施等 支持...
澳門,澳門 號巴士 由海洋花園往返九澳村嘅巴士線 背景 澳巴前身 海島市公共汽車有限公司 開辦呢條...,澳門 位處亞熱帶 下雪等寒帶地區冬季現象 在澳門極為罕見 但在過去 多年的氣象紀錄中 沿革 ...
廣東省博物館,廣東省博物館 廣東省博物館 係廣東廣州嘅省級綜合博物館 響珠江新城 歷史 文明路舊館 廣東省...,NaN


In [ ]:
pdd1 = pdd1.dropna(subset=["zh"])
pdd1 #5140 rows

In [ ]:
converter = opencc.OpenCC('s2hk.json') # Convert texts to Traditioanl HK Chinese if any in the ZH column

def script_converter(x):
  x1 = converter.convert(x)
  return x1

pdd1["zh_converted"] = pdd1["zh"].apply(script_converter)

In [ ]:
pdd1

In [118]:
import numpy as np
import pandas as pd

def df_to_csv(some_df, save_as="zhyue_comparable_wiki.csv"):
    return pd.DataFrame(some_df).to_csv(save_as, index=False, encoding='utf-8')

def df_to_txt(some_df, lang="yue", save_as="comparable_wiki.txt"):
    """ Save the training and test set as .txt to make a training/test set """
    return pd.DataFrame(some_df[lang]).to_csv(save_as,index=False, encoding='utf-8', header=False)

In [119]:
df_to_csv(pdd1, "zhyue_comparable_wiki.csv")
df_to_txt(pdd1, "yue", "yue_comparable_wiki.txt")
df_to_txt(pdd1, "zh", "zh_comparable_wiki.txt")

In [120]:
pdd1.to_pickle("ComparableWIKICorpus-yue-zh.pkl")
# unpickled_df = pd.read_pickle("./dummy.pkl")

In [ ]:
# pickle them
import pickle

with open('wiki_corpus.pkl', 'wb') as f:
    pickle.dump(mylist, f)

In [121]:
import pickle

# load the pickled files
with open('ComparableWIKICorpus-yue-zh.pkl', 'rb') as f:
   myDF = pickle.load(f)

myDF

,yue,zh,zh_converted
香港,香港 年度授勳同嘉獎名單 號喺憲報刊登 共有 人授勳同嘉獎 授勳典禮 號同 號喺禮賓府舉行 ...,香港 年度授勳及嘉獎名單 日於憲報刊登 授勳及嘉獎名單 大紫荊勳章 張舉能首席法官 鄭若驊司...,香港 年度授勳及嘉獎名單 日於憲報刊登 授勳及嘉獎名單 大紫荊勳章 張舉能首席法官 鄭若驊司...
愛因斯坦,愛因斯坦 全名音譯 阿爾拔 愛因斯坦 係一位理論物理學家 佢最出名嘅係發表咗相對論 另外喺量...,愛因斯坦 嘉當理論 此理論以物理學家阿爾伯特 愛因斯坦以及埃利 嘉當 為名 作為古典物理中的...,愛因斯坦 嘉當理論 此理論以物理學家阿爾伯特 愛因斯坦以及埃利 嘉當 為名 作為古典物理中的...
北京,北京 普通話拼音 běijīng 音標 peɪ ɕiŋ 傳統英文名 peking 簡稱 係中...,北京 年冬残奥会代表团团长会召开 代表团团长会期间 北京冬奥组委体育 场馆与基础设施等 支持...,北京 年冬殘奧會代表團團長會召開 代表團團長會期間 北京冬奧組委體育 場館與基礎設施等 支持...
澳門,澳門 號巴士 由海洋花園往返九澳村嘅巴士線 背景 澳巴前身 海島市公共汽車有限公司 開辦呢條...,澳門 位處亞熱帶 下雪等寒帶地區冬季現象 在澳門極為罕見 但在過去 多年的氣象紀錄中 沿革 ...,澳門 位處亞熱帶 下雪等寒帶地區冬季現象 在澳門極為罕見 但在過去 多年的氣象紀錄中 沿革 ...
老夫子,老夫子 英文名 old master 可以指兩種作品 第一種係王澤嘅 老夫子 年出版 佢係一...,老夫子 呈現了 老夫子 年開始在香港的報章雜誌刊載 隨後於 受到香港及各地華人喜愛 在台灣 ...,老夫子 呈現了 老夫子 年開始在香港的報章雜誌刊載 隨後於 受到香港及各地華人喜愛 在台灣 ...
...,...,...,...
童你一起長大了,童你一起長大了 係香港無綫電視嘅清談節目 由譚玉瑛 黎芷珊 蓋世寶主持 用無綫經典細路節目做...,童你一起長大了 是香港無綫電視之懷舊清談節目 由譚玉瑛 黎芷珊 蓋世寶主持 日至 日周日 翡...,童你一起長大了 是香港無綫電視之懷舊清談節目 由譚玉瑛 黎芷珊 蓋世寶主持 日至 日週日 翡...
賢學思政,賢學思政 喺旺角設置街站 之後畀警員截查同票控違反限聚令 賢學思政 係香港一個本土派學生組織...,賢學思政 在旺角設置街站 之後被警員截查和票控違反限聚令 賢學思政 是香港本土派學生組織 成...,賢學思政 在旺角設置街站 之後被警員截查和票控違反限聚令 賢學思政 是香港本土派學生組織 成...
拉闊音樂會,拉闊音樂會 風火雷電 係香港商業電台叱咤 主辦嘅演唱音樂會 由林家謙 姜濤 柳應廷 程浚彥 ...,拉闊音樂會 是商業電台叱咤 年開始舉辦的流行音樂會系列 每年舉辦數場 邀請不同歌手擔演 拉闊...,拉闊音樂會 是商業電台叱吒 年開始舉辦的流行音樂會系列 每年舉辦數場 邀請不同歌手擔演 拉闊...
港鐵機場鐵路列車,港鐵機場鐵路列車 簡稱或 係港鐵公司 前稱地鐵公司 為東涌綫同機場快綫買嘅列車 呢款列車由西...,港鐵機場鐵路列車 簡稱或 是港鐵公司 前稱地鐵公司 該列車由西班牙的caf 以及德國與瑞士的...,港鐵機場鐵路列車 簡稱或 是港鐵公司 前稱地鐵公司 該列車由西班牙的caf 以及德國與瑞士的...


## Bitext mining (Mine parallel sentences)

### Environment setup

#### One-time enviornment setup for conda

In [ ]:
#Download and install miniconda to /content/miniconda3 directory:
%env PYTHONPATH=
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.9.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.9.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.9.2-Linux-x86_64.sh -b -f -p /content/miniconda3

In [ ]:
#Add miniconda to the system PATH:
import os
path = '/content/miniconda3/bin:' + os.environ['PATH']
%env PATH=$path

env: PATH=/content/miniconda3/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin


In [ ]:
# install faiss (similarity search) via conda

!conda install -c pytorch faiss-gpu # GPU(+CPU) version

In [ ]:
#print the version of the packagexyz and its location within the conda directory
import sys
_ = sys.path.append("/content/miniconda3/lib/python3.7/site-packages")
import faiss
print(faiss.__version__, faiss.__file__)

1.7.1 /content/miniconda3/lib/python3.7/site-packages/faiss/__init__.py


In [ ]:
#Copy everything over to Google Drive
!tar -zcf conda_colab.tar.gz miniconda3
!cp conda_colab.tar.gz /content/drive/MyDrive/

#### Copy conda back to Colab (run whenever restarting a notebook)

In [ ]:
# copy back the conda installation, and re-setup the environment:
#from google.colab import drive 
#drive.mount('/content/drive')

!tar -xf /content/drive/MyDrive/conda_colab.tar.gz -C ../

import os
path = '/content/miniconda3/bin:' + os.environ['PATH']
%env PATH=$path
%env PYTHONPATH=
import sys
_ = sys.path.append("/content/miniconda3/lib/python3.7/site-packages")

#### Set up LASER

In [ ]:
!git clone https://github.com/facebookresearch/LASER

In [18]:
#set the environment variable 'LASER' to the root of the installation
import os
os.environ['LASER'] = "LASER"

In [17]:
!setenv  LASER /content/projects/laser

/bin/bash: setenv: command not found


In [19]:
!export | grep LASER

declare -x LASER="LASER"


In [ ]:
# download encoders from Amazon s3
!bash /content/LASER/install_models.sh

In [ ]:
#download third party software
!bash /content/LASER/install_external_tools.sh

In [20]:
#  calculate sentence embeddings for a text file
# The input will be tokenized, using the mode of the specified language, 
# BPE will be applied and the sentence embeddings will be calculated.
!bash /content/LASER/source/embed.py --encoder bilstm.93langs.2018-12-26.pt --bpe-codes 93langs.fcodes --token-lang yue  --verbose --output yue_embeddings.raw

/content/LASER/source/embed.py: line 19: import: command not found
/content/LASER/source/embed.py: line 20: import: command not found
/content/LASER/source/embed.py: line 21: import: command not found
/content/LASER/source/embed.py: line 22: import: command not found
/content/LASER/source/embed.py: line 23: import: command not found
/content/LASER/source/embed.py: line 24: import: command not found
/content/LASER/source/embed.py: line 25: import: command not found
from: can't read /var/mail/collections
/content/LASER/source/embed.py: line 28: import: command not found
/content/LASER/source/embed.py: line 29: import: command not found
/content/LASER/source/embed.py: line 32: syntax error near unexpected token `('
/content/LASER/source/embed.py: line 32: `assert os.environ.get('LASER'), 'Please set the enviornment variable LASER''


In [ ]:
# The embeddings are stored in float32 matrices in raw binary format. 
# They can be read in Python by:
import numpy as np
dim = 1024
X = np.fromfile("yue_embeddings.raw", dtype=np.float32, count=-1)                                                                          
X.resize(X.shape[0] // dim, dim) # X is a N x 1024 matrix where N is the number of lines in the text file.                                              

In [ ]:
## a joint sentence embedding for all the considered languages

In [ ]:
!python /content/LASER/source/mine_bitexts.py \
  zh_para yue_para \
  --src-lang="zh" --trg-lang="yue" \
  --output mined.out
  --mode mine
  --verbose
  # --gpu

In [ ]:
!git clone https://github.com/pytorch/fairseq

In [ ]:
# frequency cleaning
!wget https://dl.fbaipublicfiles.com/m2m_100/histograms.tar.gz 
!tar -xvzf histograms.tar.gz
!python /content/fairseq/examples/m2m_100/process_data/clean_histogram.py --src "zh" --tgt "yue" --src-file zh_comparable_wiki.txt --tgt-file yue_comparable_wiki.txt  --src-output-file source_output.zh --tgt-output-file target_output.yue --histograms /content/histograms

In [ ]:
# apply SPM
!wget https://dl.fbaipublicfiles.com/m2m_100/spm.128k.model
!python /content/fairseq/scripts/spm_encode.py \
    --model spm.128k.model \
    --output_format=piece \
    --inputs=/path/to/input/file/here \
    --outputs=/path/to/output/file/here

# length ratio cleaning
!perl mosesdecoder/scripts/training/clean-corpus-n.perl --ratio 3 /path/to/training/data/train.spm.$src-$tgt $src $tgt /path/to/output/directory/train.spm.$src-$tgt 1 250

# binarize data
!wget https://dl.fbaipublicfiles.com/m2m_100/data_dict.128k.txt
fairseq-preprocess \
    --source-lang $src --target-lang $tgt \
    --testpref spm.$src.$tgt \
    --thresholdsrc 0 --thresholdtgt 0 \
    --destdir data_bin \
    --srcdict data_dict.128k.txt --tgtdict data_dict.128k.txt